In [94]:
import os
import cv2
import sys
import shutil
import random
import numpy as np
faces_dir = 'att_faces'
faces_dir
faces_count=40
energy = 0.85
train_faces_count = 6                                                       # number of faces used for training
test_faces_count = 4
l=train_faces_count*faces_count
m=92
n=112
mn=m*n
L=np.empty(shape=(mn,l),dtype='float64')
cur_img=0
for face_id in range(1,faces_count+1):
    training_ids=random.sample(range(1,11),train_faces_count)
    #training_ids.append(training_ids)
    for training_id in training_ids:
        path_to_img = os.path.join(faces_dir,'s' + str(face_id), str(training_id) + '.pgm')
        img=cv2.imread(path_to_img,0)
        img_col=np.array(img,dtype='float64').flatten()
        L[:, cur_img] = img_col[:]
        
        cur_img+=1
        
        #print(l)
mean_img_col = np.sum(L, axis=1) / l
for j in range(0, l):                                             # subtract from all training images
    L[:, j] -= mean_img_col[:]
    

C = np.matrix(L.transpose()) * np.matrix(L)                             # instead of computing the covariance matrix as
C /= l  

evalues, evectors = np.linalg.eig(C)                          # eigenvectors/values of the covariance matrix
sort_indices = evalues.argsort()[::-1]                             # getting their correct order - decreasing
evalues = evalues[sort_indices]                               # puttin the evalues in that order
evectors = evectors[sort_indices] 
print(evectors.shape)
# same for the evectors


evalues_sum = sum(evalues[:])
evalues_count = 0
evalues_energy = 0.0
for evalue in evalues:
    evalues_count += 1
    evalues_energy += evalue / evalues_sum
    if evalues_count >= 220:
        break
        
evalues = evalues[0:evalues_count]
evectors = evectors[0:evalues_count]
evectors = evectors.transpose()
print(evectors.shape)
evectors = L *evectors
print(evectors.shape)
norms = np.linalg.norm(evectors, axis=0) 
evectors = evectors / norms 
W = evectors.transpose() * L 
print ('> Initializing ended')
#print(path_to_img)        
#print(L)

(240, 240)
(240, 220)
(10304, 220)
> Initializing ended


In [95]:
from PIL import Image
W.shape
M=evectors.dot(W)
M.shape
ig=M[:,0]

v=ig.reshape(112,92)
v.shape
img = Image.fromarray(v)
img.show()
if img.mode != 'RGB':
    img = img.convert('RGB')
    
img.save('my.png')

In [73]:
path_to_img
img=cv2.imread("att_faces/s5/5.pgm",0)
img
img_col=np.array(img,dtype='float64').flatten()
img_col -= mean_img_col 
img_col = np.reshape(img_col, (mn, 1))
print(img_col)
S = evectors.transpose() * img_col
diff = W - S                                                       # finding the min ||W_j - S||
norms = np.linalg.norm(diff, axis=0)
closest_face_id = np.argmin(norms)
print (closest_face_id/train_faces_count +1)

[[ 68.56666667]
 [ 67.61666667]
 [ 67.0625    ]
 ..., 
 [ 35.15416667]
 [ 33.54583333]
 [ 40.3625    ]]
6.33333333333


In [86]:
path_to_img = os.path.join(faces_dir,'s' + str(face_id), str(training_id) + '.pgm')
img=cv2.imread(path_to_img,0)
img.shape

(112, 92)